This notebook shows examples of running different queries on the engines Tumult, OpenDP and GoogleDP

Below we initiate some helper functions and get an access token

In [ ]:
from requests import request
from pprint import pprint

SERVER_API_ROOT = "http://localhost:8000/v2"
ROOT_USER = {"username": "root", "password": "123"}


def make_request(endpoint, method, token=None, body=None, content_type="application/json"):
    headers = { "Content-Type": content_type }
    if token is not None:
        headers["Authorization"] = f"Bearer {token}"
    request_args={
      "url": f"{SERVER_API_ROOT}{endpoint}",
      "method": method,
      "headers": headers,
    }
    if content_type == "application/json" and body is not None:
      request_args["json"] = body
    if content_type == "text/csv" and body is not None:
      request_args["data"] = body

    return request(**request_args)

response = make_request(endpoint="/login", method="POST", body=ROOT_USER).json()

TOKEN = response['token']

Below we create a dataset

In [ ]:
create_dataset_body = {
    "name": "salaries",
    "owner": "root",
    "schema": [
        { "name": "name",   "type": { "name": "Text" } },
        { "name": "age",    "type": { "name": "Int", "low": 18, "high": 100 } },
        { "name": "job",    "type": { "name": "Enum", "labels": ["Accountant", "Dentist", "High School Teacher", "Software Engineer"] } },
        { "name": "salary", "type": { "name": "Int", "low": 0, "high": 100000 } }
    ],
    "privacy_notion": "PureDP",
    "total_budget": { "epsilon": 50 }
}


response = make_request(endpoint="/datasets", method="POST", token=TOKEN, body=create_dataset_body)
pprint(response.json())

Below we upload data

In [ ]:
with open("demo_data.csv") as csv:
    data = csv.read()

response = make_request(endpoint="/datasets/1/upload", method="POST", token=TOKEN, content_type="text/csv", body=data.encode())
pprint(response)

Below we allocate budget on the dataset for the user

In [ ]:
allocate_budget_body = {"epsilon":50}

response = make_request(endpoint="/budgets/allocations/root/1", method="POST", token=TOKEN, body=allocate_budget_body)
pprint(response)

Below we do eval queries for different engine

In [ ]:
TUM_EVAL = "?engine=tumult"
OPEN_EVAL = "?engine=opendp"
GDP_EVAL = "?engine=googledp"

Below we showcase how similar queries are run on the three different engines, for the most part in the below section all three engines support each of the query type except for min, max

Count

In [ ]:
# GoogleDP Count
GDP_COUNT_BODY = {
    "budget": {
        "epsilon": 1
    },
    "dataset": 1,
    "query": [
        {
            "count": {
                "column": "age",
                "mech": "Laplace"
            }
        }
    ]
}

gcd_resp = make_request(endpoint=f'/queries/evaluate{GDP_EVAL}', method="POST", token=TOKEN, body=GDP_COUNT_BODY)
pprint(gcd_resp.json())

In [ ]:
# OpenDP Count
ODP_COUNT_BODY = {
    "budget": {
        "epsilon": 1
    },
    "dataset": 1,
    "query": [
        {
            "count": {
                "column": "age",
                "mech": "Laplace"
            }
        }
    ]
}

ocd_resp = make_request(endpoint=f'/queries/evaluate{OPEN_EVAL}', method="POST", token=TOKEN, body=ODP_COUNT_BODY)
pprint(ocd_resp.json())

In [ ]:
# Tumult Count
TUM_COUNT_BODY = {
    "budget": {
        "epsilon": 1
    },
    "dataset": 1,
    "query": [
        {
            "count": {
                "mech": "Laplace"
            }
        }
    ]
}

tcb_resp = make_request(endpoint=f'/queries/evaluate{TUM_EVAL}', method="POST", token=TOKEN, body=TUM_COUNT_BODY)
pprint(tcb_resp.json())

Sum

In [ ]:
# GoogleDP sum
GDP_SUM_BODY = {
    "budget": {
        "epsilon": 1
    },
    "dataset": 1,
    "query": [
        {
            "sum": {
                "column": "age",
                "mech": "Laplace"
            }
        }
    ]
}

gsd_resp = make_request(endpoint=f'/queries/evaluate{GDP_EVAL}', method="POST", token=TOKEN, body=GDP_SUM_BODY)
pprint(gsd_resp.json())


In [ ]:
# OpenDP Sum
ODP_SUM_BODY = {
    "budget": {
        "epsilon": 1
    },
    "dataset": 1,
    "query": [
        {
            "sum": {
                "column": "age",
                "mech": "Laplace"
            }
        }
    ]
}

osd_resp = make_request(endpoint=f'/queries/evaluate{OPEN_EVAL}', method="POST", token=TOKEN, body=ODP_SUM_BODY)
pprint(osd_resp.json())

In [ ]:
# Tumult Sum
TUM_SUM_BODY = {
    "budget": {
        "epsilon": 0.5
    },
    "dataset": 1,
    "query": [
        {
            "sum": {
                "column": "age"
            }
        }
    ]
}

tsb_resp = make_request(endpoint=f'/queries/evaluate{TUM_EVAL}', method="POST", token=TOKEN, body=TUM_SUM_BODY)
pprint(tsb_resp.json())

Mean

In [ ]:
# GoogleDP mean
GDP_MEAN_BODY = {
    "budget": {
        "epsilon": 1
    },
    "dataset": 1,
    "query": [
        {
            "mean": {
                "column": "age",
                "mech": "Laplace"
            }
        }
    ]
}

gmd_resp = make_request(endpoint=f'/queries/evaluate{GDP_EVAL}', method="POST", token=TOKEN, body=GDP_MEAN_BODY)
pprint(gmd_resp.json())

In [ ]:
# OpenDP Mean Not Supported
# Recommended to do post processing using results from count and sum queries


In [ ]:
# Tumult Mean
TUM_MEAN_BODY = {
    "budget": {
        "epsilon": 1
    },
    "dataset": 1,
    "query": [
        {
            "mean": {
                "column": "age"
            }
        }
    ]
}

tmb_resp = make_request(endpoint=f'/queries/evaluate{TUM_EVAL}', method="POST", token=TOKEN, body=TUM_MEAN_BODY)
pprint(tmb_resp.json())

Min

In [ ]:
# GoogleDP min not supported
# OpenDP min not supported

# Tumult Min
TUM_MIN_BODY = {
    "budget": {
        "epsilon": 1
    },
    "dataset": 1,
    "query": [
        {
            "min": {
                "column": "age"
            }
        }
    ]
}

tminb_resp = make_request(endpoint=f'/queries/evaluate{TUM_EVAL}', method="POST", token=TOKEN, body=TUM_MIN_BODY)
pprint(tminb_resp.json())

Max

In [ ]:
# GoogleDP max not supported
# OpenDP max not supported

# Tumult Max
TUM_MAX_BODY = {
    "budget": {
        "epsilon": 1
    },
    "dataset": 1,
    "query": [
        {
            "max": {
                "column": "age"
            }
        }
    ]
}

tmaxb_resp = make_request(endpoint=f'/queries/evaluate{TUM_EVAL}', method="POST", token=TOKEN, body=TUM_MAX_BODY)
pprint(tmaxb_resp.json())

Filter + Count


In [ ]:
# GoogleDP Filter + Count
GDP_FILTER_COUNT_BODY = {
    "budget": {
        "epsilon": 1
    },
    "dataset": 1,
    "query": [
        {
            "filter": ["age > 20", "age < 60"]
        },
        {
            "count": {
                "column": "age",
                "mech": "Laplace"
            }
        }
    ]
}

gfcd_resp = make_request(endpoint=f'/queries/evaluate{GDP_EVAL}', method="POST", token=TOKEN, body=GDP_FILTER_COUNT_BODY)
pprint(gfcd_resp.json())

In [ ]:
# OpenDP

In [ ]:
# Tumult Filter + Count
TUM_FILTER_COUNT_BODY = {
    "budget": {
        "epsilon": 1
    },
    "dataset": 1,
    "query": [
        { 
            "filter": ["age > 20", "age < 60"]
        },
        {
            "count": {
                "mech": "Laplace"
            }
        }
    ]
}

tfcb_resp = make_request(endpoint=f'/queries/evaluate{TUM_EVAL}', method="POST", token=TOKEN, body=TUM_FILTER_COUNT_BODY)
pprint(tfcb_resp.json())

Filter + Sum

In [ ]:
# GoogleDP Filter + Sum
GDP_FILTER_SUM_BODY = {
    "budget": {
        "epsilon": 1
    },
    "dataset": 1,
    "query": [
        {
            "filter": ["age > 20", "age < 60"]
        },
        {
            "sum": {
                "column": "age",
                "mech": "Laplace"
            }
        }
    ]
}

gfsd_resp = make_request(endpoint=f'/queries/evaluate{GDP_EVAL}', method="POST", token=TOKEN, body=GDP_FILTER_SUM_BODY)
pprint(gfsd_resp.json())

In [ ]:
# OpenDP

In [ ]:
# Tumult Filter + Sum
TUM_FILTER_SUM_BODY = {
    "budget": {
        "epsilon": 1
    },
    "dataset": 1,
    "query": [
        { 
            "filter": ["age > 20", "age < 60"] 
        },
        {
            "sum": {
                "column": "age",
                "mech": "Laplace"
            }
        }
    ]
}

tfsb_resp = make_request(endpoint=f'/queries/evaluate{TUM_EVAL}', method="POST", token=TOKEN, body=TUM_FILTER_SUM_BODY)
pprint(tfsb_resp.json())

Filter + Mean

In [ ]:
# GoogleDP Filter + Mean
GDP_FILTER_MEAN_BODY = {
    "budget": {
        "epsilon": 1
    },
    "dataset": 1,
    "query": [
        {
            "filter": ["age > 20", "age < 60"]
        },
        {
            "mean": {
                "column": "age",
                "mech": "Laplace"
            }
        }
    ]
}

gfmd_resp = make_request(endpoint=f'/queries/evaluate{GDP_EVAL}', method="POST", token=TOKEN, body=GDP_FILTER_MEAN_BODY)
pprint(gfmd_resp.json())

In [ ]:
# OpenDP

In [ ]:
# Tumult Filter + Mean
TUM_FILTER_MEAN_BODY = {
    "budget": {
        "epsilon": 1
    },
    "dataset": 1,
    "query": [
        { 
            "filter": ["age > 20", "age < 60"] 
        },
        {
            "mean": {
                "column": "age",
                "mech": "Laplace"
            }
        }
    ]
}

tfmb_resp = make_request(endpoint=f'/queries/evaluate{TUM_EVAL}', method="POST", token=TOKEN, body=TUM_FILTER_MEAN_BODY)
pprint(tfmb_resp.json())

Bin + Count

In [ ]:
# GoogleDP Bin + Count
GDP_BIN_COUNT_BODY = {
    "budget": {
        "epsilon": 1
    },
    "dataset": 1,
    "query": [
        {
            "bin": {
                "age": [20,30,40,50,60]
            }
        },
        {
            "count": {
                "column": "age",
                "mech": "Laplace"
            }
        }
    ]
}

gbcd_resp = make_request(endpoint=f'/queries/evaluate{GDP_EVAL}', method="POST", token=TOKEN, body=GDP_BIN_COUNT_BODY)
pprint(gbcd_resp.json())

In [ ]:
# OpenDP

In [ ]:
# Tumult Bin + Count
TUM_BIN_COUNT_BODY = {
    "budget": {
        "epsilon": 1
    },
    "dataset": 1,
    "query": [
        { 
            "bin": {
                "age": [20, 30, 40, 50, 60]
            }
        },
        { 
            "groupby":  {
                    "age_binned": [30, 40, 50, 60],
            }
        },
        {
            "count": {
                "mech": "Laplace"
            }
        }
    ]
}

tbcb_resp = make_request(endpoint=f'/queries/evaluate{TUM_EVAL}', method="POST", token=TOKEN, body=TUM_BIN_COUNT_BODY)
pprint(tbcb_resp.json())

Bin + Sum

In [ ]:
# GoogleDP Bin + Sum
GDP_BIN_SUM_BODY = {
    "budget": {
        "epsilon": 1
    },
    "dataset": 1,
    "query": [
        {
            "bin": {
                "age": [20,30,40,50,60]
            }
        },
        {
            "sum": {
                "column": "age",
                "mech": "Laplace"
            }
        }
    ]
}

gbsd_resp = make_request(endpoint=f'/queries/evaluate{GDP_EVAL}', method="POST", token=TOKEN, body=GDP_BIN_SUM_BODY)
pprint(gbsd_resp.json())

In [ ]:
# OpenDP

In [ ]:
# Tumult Bin + Sum
TUM_BIN_SUM_BODY = {
    "budget": {
        "epsilon": 1
    },
    "dataset": 1,
    "query": [
        { 
            "bin": {
                "age": [20, 30, 40, 50, 60]
            }
        },
        { 
            "groupby":  {
                    "age_binned": [30, 40, 50, 60],
            }
        },
        {
            "sum": {
                "column" : "age",
                "mech": "Laplace"
            }
        }
    ]
}

tbsb_resp = make_request(endpoint=f'/queries/evaluate{TUM_EVAL}', method="POST", token=TOKEN, body=TUM_BIN_SUM_BODY)
pprint(tbsb_resp.json())

Bin + Mean

In [ ]:
# GoogleDP Bin + Mean
GDP_BIN_MEAN_BODY = {
    "budget": {
        "epsilon": 1
    },
    "dataset": 1,
    "query": [
        {
            "bin": {
                "age": [20,30,40,50,60]
            }
        },
        {
            "mean": {
                "column": "age",
                "mech": "Laplace"
            }
        }
    ]
}

gbmd_resp = make_request(endpoint=f'/queries/evaluate{GDP_EVAL}', method="POST", token=TOKEN, body=GDP_BIN_MEAN_BODY)
pprint(gbmd_resp.json())

In [ ]:
# OpenDP

In [ ]:
# Tumult Bin + Mean
TUM_BIN_MEAN_BODY = {
    "budget": {
        "epsilon": 1
    },
    "dataset": 1,
    "query": [
        { 
            "bin": {
                "age": [20, 30, 40, 50, 60]
            }
        },
        { 
            "groupby":  {
                    "age_binned": [30, 40, 50, 60],
            }
        },
        {
            "mean": {
                "column" : "age",
                "mech": "Laplace"
            }
        }
    ]
}

tbmb_resp = make_request(endpoint=f'/queries/evaluate{TUM_EVAL}', method="POST", token=TOKEN, body=TUM_BIN_MEAN_BODY)
pprint(tbmb_resp.json())

Filter + Bin + Count

In [ ]:
# GoogleDP Filter + Bin + Count
GDP_FILTER_BIN_COUNT_BODY = {
    "budget": {
        "epsilon": 1
    },
    "dataset": 1,
    "query": [
        {
            "filter": ["age > 20", "age < 60"]
        },
        {
            "bin": {
                "age": [20,30,40,50,60]
            }
        },
        {
            "count": {
                "column": "age",
                "mech": "Laplace"
            }
        }
    ]
}

gfbcb_resp = make_request(endpoint=f'/queries/evaluate{GDP_EVAL}', method="POST", token=TOKEN, body=GDP_FILTER_BIN_COUNT_BODY)
pprint(gfbcb_resp.json())

In [ ]:
# OpenDP

In [ ]:
# Tumult
TUM_FILTER_BIN_COUNT_BODY = {
    "budget": {
        "epsilon": 1
    },
    "dataset": 1,
    "query": [
        { 
            "filter": ["age > 20", "age < 60"]
        },
        { 
            "bin": {
                "age": [20, 30, 40, 50, 60]
            }
        },
        { 
            "groupby":  {
                    "age_binned": [30, 40, 50, 60],
            }
        },
        {
            "count": {
                "mech": "Laplace"
            }
        }
    ]
}

tfbcb_resp = make_request(endpoint=f'/queries/evaluate{TUM_EVAL}', method="POST", token=TOKEN, body=TUM_FILTER_BIN_COUNT_BODY)
pprint(tfbcb_resp.json())

Filter + Bin + Sum

In [ ]:
# GoogleDP Filter + Bin + Sum
GDP_FILTER_BIN_SUM_BODY = {
    "budget": {
        "epsilon": 1
    },
    "dataset": 1,
    "query": [
        {
            "filter": ["age > 20", "age < 60"]
        },
        {
            "bin": {
                "age": [20,30,40,50,60]
            }
        },
        {
            "sum": {
                "column": "age",
                "mech": "Laplace"
            }
        }
    ]
}

gfbsb_resp = make_request(endpoint=f'/queries/evaluate{GDP_EVAL}', method="POST", token=TOKEN, body=GDP_FILTER_BIN_SUM_BODY)
pprint(gfbsb_resp.json())

In [ ]:
# OpenDP

In [ ]:
# Tumult Filter + Bin + Sum
TUM_FILTER_BIN_SUM_BODY = {
    "budget": {
        "epsilon": 1
    },
    "dataset": 1,
    "query": [
        { 
            "filter": ["age > 20", "age < 60"]
        },
        { 
            "bin": {
                "age": [20, 30, 40, 50, 60]
            }
        },
        { 
            "groupby":  {
                    "age_binned": [30, 40, 50, 60],
            }
        },
        {
            "sum": {
                "column": "age",
                "mech": "Laplace"
            }
        }
    ]
}

tfbsb_resp = make_request(endpoint=f'/queries/evaluate{TUM_EVAL}', method="POST", token=TOKEN, body=TUM_FILTER_BIN_SUM_BODY)
pprint(tfbsb_resp.json())

Filter + Bin + Mean

In [ ]:
# GoogleDP Filter + Bin + Mean
GDP_FILTER_BIN_MEAN_BODY = {
    "budget": {
        "epsilon": 1
    },
    "dataset": 1,
    "query": [
        {
            "filter": ["age > 20", "age < 60"]
        },
        {
            "bin": {
                "age": [20,30,40,50,60]
            }
        },
        {
            "mean": {
                "column": "age",
                "mech": "Laplace"
            }
        }
    ]
}

gfbmb_resp = make_request(endpoint=f'/queries/evaluate{GDP_EVAL}', method="POST", token=TOKEN, body=GDP_FILTER_BIN_MEAN_BODY)
pprint(gfbmb_resp.json())

In [ ]:
# OpenDP

In [ ]:
# Tumult Filter + Bin + Mean
TUM_FILTER_BIN_MEAN_BODY = {
    "budget": {
        "epsilon": 1
    },
    "dataset": 1,
    "query": [
        { 
            "filter": ["age > 20", "age < 60"] 
        },
        { 
            "bin": {
                "age": [20, 30, 40, 50, 60]
            }
        },
        { 
            "groupby":  {
                    "age_binned": [30, 40, 50, 60],
            }
        },
        {
            "mean": {
                "column": "age",
                "mech": "Laplace"
            }
        }
    ]
}

tfbmb_resp = make_request(endpoint=f'/queries/evaluate{TUM_EVAL}', method="POST", token=TOKEN, body=TUM_FILTER_BIN_MEAN_BODY)
pprint(tfbmb_resp.json())

Below queries are only supported in Tumult at the moment

In [ ]:
# Tumult GroupBy + Mean
TUM_GBY_MEAN_BODY = {
        "dataset": 1,
        "budget": { "epsilon": 0.5 },
        "query": [
            { "groupby": { "job": ["Accountant", "Dentist", "High School Teacher", "Software Engineer"] } },
            { "sum": { "column": "salary" } }
        ]
    }

tgmb_resp = make_request(endpoint=f'/queries/evaluate{TUM_EVAL}', method="POST", token=TOKEN, body=TUM_GBY_MEAN_BODY)
pprint(tgmb_resp.json())

In [ ]:
# Tumult Bin + GroupBy + Mean
TUM_BIN_GBY_MEAN_BODY = {
        "dataset": 1,
        "budget": { "epsilon": 1 },
        "query": [
            { "bin": { "age": [18, 30, 45, 60, 75] } },
            { "groupby":  {
                    "age_binned": [30, 45, 60, 75],
                    "job": ["Accountant", "Dentist", "High School Teacher", "Software Engineer"]
                }
            },
            { "mean": { "column": "salary" } }
        ]
    }

tbgmb_resp = make_request(endpoint=f'/queries/evaluate{TUM_EVAL}', method="POST", token=TOKEN, body=TUM_BIN_GBY_MEAN_BODY)
pprint(tbgmb_resp.json())